# 02_03_solving_fits_and_making_sh

## 필요한 환경

이 프로젝트를 위해서는 아래의 모듈이 필요합니다.

> numpy, pandas, matplotlib, scipy, astropy, photutils, ccdproc, ysfitsutilpy, ysphotutilpy, version_information

### 모듈 버전 확인

아래 셀을 실행하면 이 노트북을 실행한 파이썬 및 관련 모듈의 버전을 확인할 수 있다.

In [ ]:
import importlib, sys, subprocess
packages = "numpy, pandas, matplotlib, scipy, astropy, ccdproc, ysfitsutilpy, ysphotutilpy, version_information" # required modules
pkgs = packages.split(", ")

for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        print(f"**** module {pkg} is not installed... now start install")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
        print(f"****** module {pkg} is installed")
    else: 
        print(f"**** module {pkg} is installed")

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

### import modules

In [ ]:
from glob import glob
from pathlib import Path
import os
from astropy.io import fits
import astropy.units as u

from astroquery.astrometry_net import AstrometryNet

import ysfitsutilpy as yfu

import _astro_utilities
import _Python_utilities

# 환경 설정

In [ ]:
#%%
#######################################################
BASEDIR = Path("/mnt/Rdata/OBS_data") 
PROJECDIR = Path("/mnt/Rdata/OBS_data/2024-EXO")
TODODIR = PROJECDIR / "_-_-_2024-05_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "_-_-_2024-06_-_GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_2bin"

PROJECDIR = Path("/mnt/Rdata/OBS_data/2022-Asteroid")
TODODIR = PROJECDIR / "GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_2bin"

PROJECDIR = Path("/mnt/Rdata/OBS_data/2023-Asteroid")
TODODIR = PROJECDIR / "GSON300_STF-8300M_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_1bin"
TODODIR = PROJECDIR / "RiLA600_STX-16803_-_2bin"

PROJECDIR = Path("/mnt/Rdata/OBS_data/2016-Variable")
TODODIR = PROJECDIR / "-_-_-_2016-_-_RiLA600_STX-16803_-_2bin"

# PROJECDIR = Path("/mnt/Rdata/OBS_data/2017-Variable")
# TODODIR = PROJECDIR / "-_-_-_2017-_-_RiLA600_STX-16803_-_2bin"

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfsubDirs(TODODIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

BDFDIR = [x for x in DOINGDIRs if "CAL-BDF" in str(x)]
print ("BDFDIR: ", format(BDFDIR))

MASTERDIR = Path(BDFDIR[0]) / _astro_utilities.master_dir
if not MASTERDIR.exists():
    os.makedirs("{}".format(str(MASTERDIR)))
    print("{} is created...".format(str(MASTERDIR)))
print ("MASTERDIR: ", format(MASTERDIR))

DOINGDIRs = sorted([x for x in DOINGDIRs if "_LIGHT_" in str(x)])
# print ("DOINGDIRs: ", format(DOINGDIRs))
# print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

# filter_str = 'HAT-P-37b_LIGHT_-_2024-06-28_-_RiLA600_STX-16803_-_2bin'
# DOINGDIRs = [x for x in DOINGDIRs if filter_str in str(x)]
# remove = 'BIAS'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'DARK'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
# remove = 'FLAT'
# DOINGDIRs = [x for x in DOINGDIRs if remove not in x]
print ("DOINGDIRs: ", DOINGDIRs)
print ("len(DOINGDIRs): ", len(DOINGDIRs))
#######################################################

In [ ]:
# DOINGDIR = Path('/mnt/Rdata/OBS_data/CCD_obs_test/M13_LIGHT_-_2022-04-02_-_FS60CB_STF-8300M_-_1bin')
# print("DOINGDIR", DOINGDIR)
# SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
# SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
# SOLVINGDIR = DOINGDIR

# summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
# if summary is not None :
#     print("len(summary):", len(summary))
#     print("summary:", summary)
#     #print(summary["file"][0])  
#     df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
#     df_light = df_light.reset_index(drop=True)
#     print("df_light:\n{}".format(df_light))
# df_light


In [ ]:
for DOINGDIR in DOINGDIRs :
    DOINGDIR = Path(DOINGDIR)
    print("DOINGDIR", DOINGDIR)
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
    SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
    SOLVINGDIR = DOINGDIR

In [ ]:
DOINGDIR = Path(DOINGDIRs[0])
print("DOINGDIR", DOINGDIR)
SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_dir
SOLVINGDIR = DOINGDIR / _astro_utilities.reduced_nightsky_dir
SOLVINGDIR = DOINGDIR

summary = yfu.make_summary(SOLVINGDIR/"*.fit*")
if summary is not None :
    print("len(summary):", len(summary))
    print("summary:", summary)
    #print(summary["file"][0])  
    df_light = summary.loc[summary["IMAGETYP"] == "LIGHT"].copy()
    df_light = df_light.reset_index(drop=True)
    print("df_light:\n{}".format(df_light))
df_light

for _, row  in df_light.iterrows():

    fpath = Path(row["file"])
    # fpath = Path(df_light["file"][1])
    print("fpath :" ,fpath)
    hdul = fits.open(fpath)

    if 'PIXSCALE' in hdul[0].header:
        PIXc = hdul[0].header['PIXSCALE']
    else : 
        PIXc = _astro_utilities.calPixScale(hdul[0].header['FOCALLEN'], 
                                            hdul[0].header['XPIXSZ'],
                                            hdul[0].header['XBINNING'])
    print("PIXc : ", PIXc)
    hdul.close()

    solved = _astro_utilities.KevinSolver(fpath, 
                                            #str(SOLVEDDIR), 
                                            # downsample = 2,
                                            # pixscale = PIXc,
                                            tryASTAP = True, 
                                            tryLOCAL = False,
                                            makeLOCALsh = True,
                                            tryASTROMETRYNET = False, 
                                            )
